### Part 3 -> Multiturn conversation dataset generation

In [1]:
import json
from tqdm import tqdm
from openai import OpenAI
import random
import pandas as pd
from multiturn_modul import MultiturnStyle, get_multiturn_style

path_to_api_key: str = "../API_KEY.txt"
my_api_key = open(path_to_api_key, 'r').read()
client = OpenAI(api_key=my_api_key)
gpt_model: str = "gpt-4o"

In [8]:
countries = pd.read_csv("../resources/countries_by_income_category.csv")
sampled_country = random.choice(countries.iloc[:, 0].tolist())
id = "0-122"

for i in range(10000):
    test = get_multiturn_style(str(i%12)+"-112", "Switzerland")

multiturn_style_1 = MultiturnStyle(id, sampled_country, nbr_of_turns=2, mode = "normal")

print(test.system_prompt_chatbot())
print(test.system_prompt_user())


You are a medical AI chatbot designed to assist health care workers working in Switzerland by answering their questions. 
The style of your answers must follow these rules: 
The user is using you (the medical AI chatbot) in chat mode. Provide short and concise answers that make it possible for the user to chat with you. Ask the user follow up questions about their question if their question is not clear enough. 
In the prompt you see the whole chat history. You are (chatbot). Your task is to answer the questions and only ask follow up questions, if the user did not provide enough information in its input. 
Your (chatbot) tag is added manually, don't write it in your output. Just generate an answer to the users input
You are a physician working in Switzerland using a medical AI chatbot to help you taking care of your patients. 
The initial question that you ask the medical AI chatbot has already been provided. You are now interacting with the medical AI chatbot and asking follow up ques

In [4]:
# Test mode
# Generate multiturn conversations between Meditron (simulated by gpt-4o) and a Physician (simulated by gpt-4o)

# System prompts
system_prompt_chatbot = multiturn_style_1.system_prompt_chatbot()
system_prompt_user = multiturn_style_1.system_prompt_user()  

# store medical AI tasks in a list
path_to_prompts = "../results/parsed_prompts_tasks_x_subtopics.json" 
with open(path_to_prompts, 'r') as file:
    task_x_subtopics_prompts = json.load(file)  # Load JSON data from file
initial_physician_prompt = task_x_subtopics_prompts[100]["prompt"]
print(initial_physician_prompt)

# Function that generates the next step of a conversation

def next_conversation_step(system_prompt, chat_histroy, verbose = False):
    if system_prompt == "user":
        content = system_prompt_user
    else:
        content = system_prompt_chatbot

    prompt = chat_histroy

    if verbose:
        print("Send prompt to GPT:")
        print("#########")
        print(prompt)
        print("########")

    completion = client.chat.completions.create(
        model= gpt_model,
        messages=[
            {"role": "system", "content": content},
            {
                "role": "user",
                "content": f"{prompt}"
            }
        ]
    )
    response = completion.choices[0].message.content
    
    if verbose:
        print("Receiving responses from GPT...")
        print(response)
    return chat_histroy + "\n" + f"({system_prompt}) " + response

A 32-year-old male patient presents with a 3-day history of high fever, chills, and body aches. He also reports a sore throat and a non-productive cough. He returned from a business trip to Southeast Asia 10 days ago. His medical history is notable for mild asthma. On examination, he has a temperature of 102°F, mild pharyngeal erythema, and bilateral cervical lymphadenopathy. Considering his recent travel history, could this presentation be indicative of an emerging infectious disease endemic to that region, and what diagnostic tests should be prioritized?


In [5]:
# Check if everything is fine
N = multiturn_style_1.number_of_turns
print(N)
print(f"chatbot: {system_prompt_chatbot}")
print(f"user: {system_prompt_user}")

2
chatbot: You are a medical AI chatbot designed to assist health care workers working in Sierra Leone by answering their questions. 
The style of your answers must follow these rules: 
Provide short and concises answers. 
Provide your answers in flat text. 
In the prompt you see the whole chat history. You are (chatbot). Your task is to answer the questions and only ask follow up questions, if the user did not provide enough information in its input. 
Your (chatbot) tag is added manually, don't write it in your output. Just generate an answer to the users input
user: You are a physician working in Sierra Leone using a medical AI chatbot to help you taking care of your patients. 
The initial question that you ask the medical AI chatbot has already been provided. You are now interacting with the medical AI chatbot and asking follow up questions, based on the answers provided by the medical AI chatbot. 
The style of the follow up question you ask must follow these rules: 
Ask short and c

In [ ]:
conversation_history = []
conversation_history.append("(user) " + initial_physician_prompt)
for i in ###tqdm(range(2*N - 1)): ### Change range -> will cost money
    if i % 2 == 0:
        conversation_history.append(next_conversation_step("chatbot", conversation_history[-1]))
    else:
        conversation_history.append(next_conversation_step("user", conversation_history[-1]))


100%|██████████| 3/3 [00:05<00:00,  1.87s/it]


In [7]:
print(conversation_history[-1])

(user) A 32-year-old male patient presents with a 3-day history of high fever, chills, and body aches. He also reports a sore throat and a non-productive cough. He returned from a business trip to Southeast Asia 10 days ago. His medical history is notable for mild asthma. On examination, he has a temperature of 102°F, mild pharyngeal erythema, and bilateral cervical lymphadenopathy. Considering his recent travel history, could this presentation be indicative of an emerging infectious disease endemic to that region, and what diagnostic tests should be prioritized?
(chatbot) Given the travel history and symptoms, possible causes could include influenza, dengue fever, or other viral infections endemic to Southeast Asia. Considering these possibilities, the following diagnostic tests should be prioritized:

1. Influenza test (rapid antigen or PCR).
2. Dengue NS1 antigen and IgM/IgG antibodies.
3. Complete Blood Count (CBC) to check for signs of viral infection.
4. Throat swab for bacterial